# 📚 Análisis Avanzado de libros.db
Este notebook realiza:
- Filtrado de libros con autor "Desconocido"
- Exportación a CSV
- Conteo de autores más frecuentes
- Reintento de búsqueda de autores vía Google Books API para los que quedaron como 'Desconocido'


In [1]:
import sqlite3
import pandas as pd
import requests

## Cargar datos desde la base `libros.db`

In [2]:
conn = sqlite3.connect("libros.db")
query = """
SELECT 
    b.title, 
    a.name as author,
    b.price,
    b.rating
FROM books b
JOIN book_authors ba ON b.id = ba.book_id
JOIN authors a ON a.id = ba.author_id
"""
df = pd.read_sql(query, conn)
df.head()


,title,author,price,rating
0,It's Only the Himalayas,S. Bedford,£45.17,Two
1,Full Moon over Noah’s Ark: An Odyssey to Mount...,Rick Antonson,£49.43,Four
2,See America: A Celebration of Our National Par...,Desconocido,£48.87,Three
3,Vagabonding: An Uncommon Guide to the Art of L...,Rolf Potts,£36.94,Two
4,Under the Tuscan Sun,Frances Mayes,£37.33,Three


## 1. Filtrar libros con autor desconocido

In [3]:
desconocidos = df[df['author'].str.lower() == 'desconocido']
print(f"Total libros con autor 'Desconocido': {len(desconocidos)}")
desconocidos.head()

Total libros con autor 'Desconocido': 505


,title,author,price,rating
2,See America: A Celebration of Our National Par...,Desconocido,£48.87,Three
12,"In a Dark, Dark Wood",Desconocido,£19.63,One
18,Tastes Like Fear (DI Marnie Rome #3),Desconocido,£10.69,One
21,Poisonous (Max Revere Novels #3),Desconocido,£26.80,Three
25,Boar Island (Anna Pigeon #19),Desconocido,£59.48,Three


## 2. Exportar a CSV todos los libros

In [4]:
df.to_csv("libros_exportados.csv", index=False)
print("✅ Exportado como libros_exportados.csv")

✅ Exportado como libros_exportados.csv


## 3. Autores más frecuentes

In [5]:
top_autores = df[df['author'].str.lower() != 'desconocido']['author'].value_counts().head(10)
top_autores

author
Natsuki Takaya      7
Yann Martel         3
Sophie Kinsella     3
Dan Brown           3
Robert Galbraith    3
Stephen King        3
Libros Maestros     3
Malcolm Gladwell    3
Cassandra Clare     3
Bill Bryson         3
Name: count, dtype: int64

## 4. Reintentar obtener autores con la API de Google Books

In [6]:
API_KEY = "AQUÍ_VA_TU_API_KEY"

def buscar_autor_google_books(titulo):
    params = {
        'q': f'intitle:{titulo}',
        'key': API_KEY,
        'maxResults': 1,
        'langRestrict': 'en'
    }
    try:
        response = requests.get("https://www.googleapis.com/books/v1/volumes", params=params)
        if response.status_code == 200:
            data = response.json()
            if "items" in data:
                volume_info = data["items"][0]["volumeInfo"]
                autores = volume_info.get("authors", ["Desconocido"])
                return ", ".join(autores)
    except:
        pass
    return "Desconocido"

In [7]:
# Reintenta para los primeros 5 como demo (cambiar a más si querés)
for i, row in desconocidos.head(5).iterrows():
    nuevo_autor = buscar_autor_google_books(row['title'])
    print(f"🔄 {row['title']} → {nuevo_autor}")

🔄 See America: A Celebration of Our National Parks & Treasured Sites → Desconocido
🔄 In a Dark, Dark Wood → Desconocido
🔄 Tastes Like Fear (DI Marnie Rome #3) → Desconocido
🔄 Poisonous (Max Revere Novels #3) → Desconocido
🔄 Boar Island (Anna Pigeon #19) → Desconocido
